In [2]:
import glob
from stempy import io, image
import numpy as np
from PIL import Image
from mpi4py import MPI
import sys
width = 513
height = 513

In [3]:
files = glob.glob('/global/cscratch1/sd/cjh1/scan_0000000308/*.dat')

In [4]:
rank=0

In [6]:
world_size=60
files_per_rank = len(files) // world_size
left_over = len(files) % world_size
if rank < left_over:
    offset = rank*(files_per_rank+1)
    files = files[offset:offset+files_per_rank+1]
else:
    offset = rank*files_per_rank+left_over
    files = files[offset:offset+files_per_rank]

In [7]:
files

['/global/cscratch1/sd/cjh1/scan_0000000308/stem4d_0000000308_0000000077.dat',
 '/global/cscratch1/sd/cjh1/scan_0000000308/stem4d_0000000308_0000000065.dat',
 '/global/cscratch1/sd/cjh1/scan_0000000308/stem4d_0000000308_0000000099.dat']

In [9]:
%%time
reader = io.reader(files, version=io.FileVersion.VERSION2)

CPU times: user 0 ns, sys: 584 µs, total: 584 µs
Wall time: 1.56 ms


In [10]:
%%time
local_stem = image.create_stem_image(reader, width, height, 40, 288)

CPU times: user 1.25 s, sys: 2.46 s, total: 3.71 s
Wall time: 3.07 s


In [30]:
local_stem.bright.shape

(513, 513)

In [31]:
local_stem.dark.shape

(513, 513)